# Redspots Environment

## Environment Description

A simple gridworld with white (safe) spots, green (rewarding) spots, and red (undesirable/dangerous) spots.

## Goal

Have an agent map the environment and infer the best way to avoid red spots and get to the green spot.

# Setup

## Imports

In [1]:
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.cm as cm

import seaborn as sns

import pymdp
from pymdp import utils

## Auxilary Functions

In [2]:
def plot_grid(grid_locations, num_x = 3, num_y = 3 ):
    """
    Plots the spatial coordinates of GridWorld as a heatmap, with each (X, Y) coordinate 
    labeled with its linear index (its `state id`)
    """

    grid_heatmap = np.zeros((num_x, num_y))
    for linear_idx, location in enumerate(grid_locations):
      y, x = location
      grid_heatmap[y, x] = linear_idx
    sns.set(font_scale=1.5)
    sns.heatmap(grid_heatmap, annot=True, cbar = False, fmt='.0f', cmap='crest')

def plot_likelihood(matrix, title_str = "Likelihood distribution (A)"):
    """
    Plots a 2-D likelihood matrix as a heatmap
    """

    if not np.isclose(matrix.sum(axis=0), 1.0).all():
      raise ValueError("Distribution not column-normalized! Please normalize (ensure matrix.sum(axis=0) == 1.0 for all columns)")
    
    fig = plt.figure(figsize = (6,6))
    ax = sns.heatmap(matrix, cmap = 'gray', cbar = False, vmin = 0.0, vmax = 1.0)
    plt.title(title_str)
    plt.show()


def plot_beliefs(belief_dist, title_str=""):
    """
    Plot a categorical distribution or belief distribution, stored in the 1-D numpy vector `belief_dist`
    """

    if not np.isclose(belief_dist.sum(), 1.0):
      raise ValueError("Distribution not normalized! Please normalize")

    plt.grid(zorder=0)
    plt.bar(range(belief_dist.shape[0]), belief_dist, color='r', zorder=3)
    plt.xticks(range(belief_dist.shape[0]))
    plt.title(title_str)
    plt.show()

def add_noise(matrix, noise_level=0.1):
    """
    Add noise to transition matrix while preserving normalization
    
    Args:
        A: Original transition matrix
        noise_level: Amount of noise to add (0-1)
    """
    # Generate random noise
    noise = np.random.uniform(-noise_level, noise_level, size=matrix.shape)

    print(noise) 
    
    # Add noise to matrix
    noisy_matrix = matrix + noise
    
    # Ensure non-negative
    noisy_matrix = np.maximum(noisy_matrix, 0.0)
    
    # Normalize columns to sum to 1
    noisy_matrix = noisy_matrix / noisy_matrix.sum(axis=0, keepdims=True)
    
    return noisy_matrix

def move_to_grid(x, y, z):
    '''Moves coppelia coordinates (x,y,z) to a 200x200 grid, z coordinate remains constant, outputs coordinate in terms of grid'''
    
    # Translate x,y coordinate 2.5 up and 2.5 right
    x = x + 2.5
    y = y + 2.5
    
    # Ensure coordinates (x,y) are within (0,0) and (5,5)
    if x > 5 or x < 0:
        return "Invalid x coordinate!"
    elif y > 5 or y < 0:
        return "Invalid y coordinate!"
    
    # Convert x, y to grid indices by dividing by 0.05 (since each grid cell is 0.05 wide)
    x_grid = round(x / 0.05)
    y_grid = round(y / 0.05)
    
    # Ensure that the coordinates are within valid grid range (0 to 200)
    if x_grid > 200 or x_grid < 0:
        return "Invalid x grid point!"
    if y_grid > 200 or y_grid < 0:
        return "Invalid y grid point!"
    
    # Return the grid indices
    return (x_grid, y_grid)

    
def grid_to_coordinates(x_grid, y_grid, z):
    '''Converts a valid 200x200 grid point back into coppelia (x,y,z) coordinates in the range (x,y) = (0,0)-(5,5), z remains constant'''
    
    # Ensure the grid points are within valid range (0 to 200)
    if x_grid > 200 or x_grid < 0:
        return "Invalid x grid point!"
    if y_grid > 200 or y_grid < 0:
        return "Invalid y grid point!"
    
    # Reverse the grid index conversion by multiplying by 0.05
    x = x_grid * 0.05
    y = y_grid * 0.05
    
    # Return the original (x, y, z) coordinates
    return (x, y, z)

## Parameters

### Grid

In [3]:
# grid_dimensions
grid_dims = [40, 40]

# list of grid positiions
grid_locations = []
for i in range(grid_dims[0]):
    for j in range(grid_dims[1]):
        grid_locations.append((i,j))

In [4]:
redspots = [(34, 20), (28, 32), (8, 28), (12, 12), (28, 8)]

In [5]:
# start point
agent_pos = (0,0)

# end goal
goal_location = (6,4)

## Generative Model

### States & Observations

s1 = current location \
s2 = attribute of current location

o1 = observed current location \
o2 = color of current location

In [6]:
# s1 is already defined = grid_locations

# s2
current_attribute = ['SAFE', 'DANGER', 'REWARDING']

# list of # of possibillities for states
num_states = [len(grid_locations), len(current_attribute)] # location

In [7]:
# o1 is already defined = grid_locatioons

# o2
current_color = ['WHITE', 'RED', 'GREEN']

# list of # of possibilities for observations
num_obs = [len(grid_locations), len(current_color)]

# Generative Model

## Agent's Observation Matrix

In [8]:
vision_matrix = np.zeros((num_states[0], num_states[1]))

# Rule-based assignment
for loc in range(num_states[0]):
    # Example: Assume all locations have [SAFE: 0.7, DANGER: 0.2, REWARDING: 0.1]
    vision_matrix[loc] = np.array([0.33, 0.33, 0.33])
    vision_matrix[loc] /= vision_matrix[loc].sum()

    # Normalize each location's attribute distribution (ensure sum = 1)

vision_matrix

In [9]:
def update_vision_matrix(vision_matrix, obs):
    """
    Update the observation matrix based on an observation
    """

    one_hot_obs = [0,0,0]
    one_hot_obs[obs[1]] = 1
    vision_matrix[obs[0]] = one_hot_obs

    return vision_matrix

In [10]:
def visualize_vision_matrix(vision_matrix, grid_dims=grid_dims):
    """
    Visualizes the observation matrix as three separate heatmaps, one for each attribute
    (SAFE, DANGER, REWARDING)
    
    Args:
        vision_matrix: numpy array of shape (num_locations, num_attributes)
        grid_dims: list of [height, width] of the grid
    """
    # Create figure with three subplots
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Labels for each subplot
    titles = ['SAFE', 'DANGER', 'REWARDING']
    
    for idx, title in enumerate(titles):
        # Reshape the matrix to match grid dimensions
        heatmap_data = vision_matrix[:, idx].reshape(grid_dims[0], grid_dims[1])
        
        # Plot heatmap
        sns.heatmap(heatmap_data, 
                   ax=axes[idx], 
                   cmap='YlOrRd',
                   vmin=0, 
                   vmax=1,
                   annot=True,
                   fmt='.2f')
        
        axes[idx].set_title(f'{title} Probability')
        axes[idx].set_xlabel('Y coordinate')
        axes[idx].set_ylabel('X coordinate')
    
    plt.tight_layout()
    plt.show()

In [11]:
visualize_vision_matrix(vision_matrix, grid_dims)

## A Matrix

In [12]:
# Define A Matrix
A_shapes = []
for i in num_obs:
    A_shapes.append([i] + num_states)

A = utils.obj_array_zeros(A_shapes)
A.shape, A_shapes

### Location Observation Modality A[0]

In [13]:
A[0] = np.zeros(A_shapes[0])  # Initialize with zeros

# Fill in the observation mapping for each safety level
for safety_level in range(num_states[1]):
    A[0][:,:,safety_level] = np.eye(num_states[0])  # Copy the identity matrix for each safety level

# Verify the shape and normalization
print("New A[0] shape:", A[0].shape)
print("Column sums:", A[0].sum(axis=0).sum(axis=1))  # Should be all 1's

In [14]:
# Extract a 2D slice by fixing the safety level dimension to 0
import numpy as np
plot_likelihood(A[0][:, :, 0], "Location observation likelihood matrix")

### Color observation modality: A[1]

In [15]:
# Map safety levels to indices
safety_level_to_index = {state: i for i, state in enumerate(current_attribute)}  # {'SAFE': 0, 'DANGER': 1, 'REWARDING': 2}

# Probabilities for each color given the safety level (in correct heatmap order: RED, GREEN, WHITE)
probabilities = {
    "SAFE": [1, 0, 0],        # ['WHITE', 'RED', 'GREEN']
    "DANGER": [0, 1, 0.],      # ['WHITE', 'RED', 'GREEN']
    "REWARDING": [0, 0, 1]    # ['WHITE', 'RED', 'GREEN']
}

In [16]:
# Populate A[1]
for safety_level, probs in probabilities.items():
    safety_idx = safety_level_to_index[safety_level]
    for loc in range(len(grid_locations)):  # Iterate over grid locations
        for color_idx, prob in enumerate(probs):  # Iterate over colors (RED, GREEN, WHITE)
            A[1][color_idx, loc, safety_idx] = prob  # Assign probabilities

In [17]:
# Create a figure with 3 subplots (one for each safety state)
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Extract and plot each safety state slice
for safety_idx in range(3):
    # Get a slice for the current safety state (fixing all other dimensions to 0)
    slice_matrix = A[1][:, :, safety_idx]
    
    # Plot the slice
    sns.heatmap(slice_matrix, ax=axes[safety_idx], cmap='gray', cbar=False, vmin=0.0, vmax=1.0)
    axes[safety_idx].set_title(f'Safety State {safety_idx}')
    axes[safety_idx].set_xlabel('Location States')
    if safety_idx == 0:
        axes[safety_idx].set_ylabel('Color Observations')

plt.tight_layout()
plt.show()

### Add Noise

In [18]:
# Add noise to each modality separately
for modality in range(len(A)):
    A[modality] = add_noise(A[modality], noise_level=0)

### Plot Each Matrix

## B Matrix

### Define Shape

In [19]:
num_controls = [5, 1]
B_f_shapes = [ [ns, ns, num_controls[f]] for f, ns in enumerate(num_states)]
B = utils.obj_array_zeros(B_f_shapes)
B_f_shapes

### B[0] - Control Factor - Location Transitions

In [20]:
grid_dims

In [21]:
actions = ["UP", "DOWN", "LEFT", "RIGHT", "STAY"]

In [22]:
for action_id, action_label in enumerate(actions):
  
    for curr_state, (x, y) in enumerate(grid_locations):
        
        # Calculate next position based on action
        if action_label == "UP":
            next_y = max(0, y - 1)        # Move up (decrease y)
            next_x = x
        elif action_label == "DOWN":
            next_y = min(grid_dims[1]-1, y + 1)  # Move down (increase y)
            next_x = x
        elif action_label == "LEFT":
            next_x = max(0, x - 1)        # Move left (decrease x)
            next_y = y
        elif action_label == "RIGHT":
            next_x = min(grid_dims[0]-1, x + 1)  # Move right (increase x)
            next_y = y
        else:  # STAY
            next_x = x
            next_y = y
        
        # Get the state index for the next position
        next_state = grid_locations.index((next_x, next_y))
        
        # Set transition probability to 1.0
        B[0][next_state, curr_state, action_id] = 1.0

### B[1] - Non-Control Factor - Identity Matrix

In [23]:
# Initialize safety level transition matrix (no changes for safety levels)
B[1][:,:,0] = np.eye(3)  # Identity matrix for safety level transitions

### Normalization

In [24]:
# Normalize B matrix columns for each action
for action_id in range(len(actions)):
    # Get slice for current action
    B_action = B[0][..., action_id]
    
    # Replace zero columns with ones in appropriate positions
    zero_cols = (B_action.sum(axis=0) == 0)
    for col in range(B_action.shape[1]):
        if zero_cols[col]:
            # Stay in the same state if no transition is defined
            B_action[col, col] = 1.0
    
    # Normalize columns
    column_sums = B_action.sum(axis=0)
    B[0][..., action_id] = B_action / column_sums[None, :]

# Verify normalization
for action_id in range(len(actions)):
    assert np.allclose(B[0][..., action_id].sum(axis=0), 1.0), f"Action {actions[action_id]} not normalized"

In [25]:
print(B[1])

## C Vectors (prior preferences)

### Initialize

In [26]:
C = utils.obj_array_zeros(num_obs)  # Initialize C array with shape matching num_obs
print(C.shape)

### C[0] - Preference for location observations

In [27]:
goal_location

In [28]:
# Set preferences for state observations (location)
C[0] = np.ones(len(grid_locations))
C[0][grid_locations.index(goal_location)] += 1

print(C[0])

In [29]:
for i, loc in enumerate(grid_locations):
    x = ((goal_location[0] - loc[0])**2 + (goal_location[1] - loc[1])**2) ** 0.5
    print(loc, i, x)
    C[0][i] -= x * 0.1

In [30]:
print(C[0])

In [31]:
from pymdp.maths import softmax

In [32]:
C[0] = softmax(C[0])

In [33]:
plot_beliefs(C[0])

### C[1] - Preference for color observations

In [34]:
# white, red, green <- order it's encoded in

In [35]:
# Set preferences for color observations
C[1] = np.zeros((num_obs[1],))
C[1][0] = -0.1
C[1][1] = -1
C[1][2] = 1.1

print(C[1])


## D Vectors: Prior beliefs about hidden states

### Initialize

In [36]:
# Shape
num_states

In [37]:
D = utils.obj_array_uniform(num_states)

### D[0] - Belief About Current Location

In [38]:
# Define prior belief about agent's location (same as before)
D[0] = np.zeros(num_states[0])  # Shape (35,)
D[0][grid_locations.index(agent_pos)] = 1.0  # One-hot encoding for location

print("D[0] shape (Location prior):", D[0].shape)  # (35,)
D[0]

In [39]:
# # Initialize uniform distribution over locations
# D[0] = np.ones(num_states[0]) / num_states[0]  # Create normalized uniform distribution over all locations

# D[0]

### D[1] - Belief About Attribute of Current Location

In [40]:
D[1] = np.ones(num_states[1]) / num_states[1]  # Create normalized uniform distribution over all locations
D[1].shape

In [41]:
plot_beliefs(D[0])

In [42]:
plot_beliefs(D[1])

# Generative Process

In [43]:
def update_vision(current_location, grid_dims, distance):
    """
    Update the agent's field of vision based on the current location and distance
    Returns a list of all grid positions within the vision range
    
    Args:
        current_location (tuple): Current (x,y) position of the agent
        grid_dims (list): Dimensions of the grid [width, height]
        distance (int): Vision range/distance
        
    Returns:
        list: List of (x,y) tuples representing visible grid positions
    """
    x, y = current_location
    x_min = max(0, x - distance)
    x_max = min(grid_dims[0], x + distance + 1)
    y_min = max(0, y - distance)
    y_max = min(grid_dims[1], y + distance + 1)
    
    visible_locations = []
    for y_pos in range(y_min, y_max):
        for x_pos in range(x_min, x_max):
            visible_locations.append((x_pos, y_pos))
            
    return visible_locations

In [44]:
X, Y = 0, 0

In [45]:
# class GridWorldEnv():
#     def __init__(self, starting_loc=(0, 0), redspots=[(1, 2), (3, 2), (4, 4), (6, 1)], goal=(6,4)):
#         # Initialize coordinates
#         self.x, self.y = starting_loc
#         self.init_loc = starting_loc
#         self.current_location = (self.x, self.y)

#         self.goal = goal

#         self.redspots = redspots

#         self.red_obs = ['Null']
#         self.green_obs = 'Null'
#         self.white_obs = ['Null']

#         self.agent_reward = 0
        
#         print(f"Starting location is {self.current_location} | Red spot locations are {self.redspots} | Goal is {self.goal}")
    
#     def step(self, action_label):
#         if action_label == "UP": 
#             self.y = max(0, self.y - 1)  # Move up (decrease y)
            
#         elif action_label == "DOWN": 
#             self.y = min(grid_dims[1] - 1, self.y + 1)  # Move down (increase y)

#         elif action_label == "LEFT": 
#             self.x = max(0, self.x - 1)  # Move left (decrease x)

#         elif action_label == "RIGHT": 
#             self.x = min(grid_dims[0] - 1, self.x + 1)  # Move right (increase x)

#         # Update current_location tuple after movement
#         self.current_location = (self.x, self.y)
#         print(f"self.current_location: {self.current_location}")
        
#         # Update vision with current coordinates
#         self.vision = update_vision(self.current_location, grid_dims, 6)

#         self.loc_obs = self.current_location

#         # Reset observations at each step
#         self.red_obs = ['Null']
#         self.white_obs = ['Null']
#         self.green_obs = 'Null'

#         # Update observations based on vision
#         for spot in self.vision:
#             if spot in self.redspots:
#                 if 'Null' in self.red_obs:
#                     self.red_obs = [spot]
#                 else:
#                     self.red_obs.append(spot)
#             elif spot == self.goal:
#                 self.green_obs = spot
#             else:
#                 if 'Null' in self.white_obs:
#                     self.white_obs = [spot]
#                 else:
#                     self.white_obs.append(spot)

#         # Update rewards and observations based on current location
#         if self.current_location in self.redspots:
#             self.agent_reward -= 5
#             if 'Null' in self.red_obs:
#                 self.red_obs = [self.current_location]
#             else:
#                 self.red_obs.append(self.current_location)
#         elif self.current_location == self.goal:
#             self.agent_reward += 20
#             self.green_obs = self.current_location
#         else:
#             if 'Null' in self.white_obs:
#                 self.white_obs = [self.current_location]
#             else:
#                 self.white_obs.append(self.current_location)
        
#         return self.loc_obs, self.green_obs, self.white_obs, self.red_obs, self.agent_reward
    
#     def reset(self):
#         self.x, self.y = self.init_loc
#         self.current_location = (self.x, self.y)
#         print(f'Re-initialized location to {self.current_location}')
#         self.loc_obs = self.current_location
#         self.green_obs, self.white_obs, self.red_obs, self.agent_reward = 'Null', ['Null'], ['Null'], 0

#         return self.loc_obs, self.green_obs, self.white_obs, self.red_obs, self.agent_reward

# Customizing Functions

In [46]:
def custom_get_expected_states(qs, B, policy, vision_matrix=vision_matrix):

    """

    Compute the expected states under a policy, also known as the posterior predictive density over states

    Parameters

    ----------

    qs: ``numpy.ndarray`` of dtype object

        Marginal posterior beliefs over hidden states at a given timepoint.

    B: ``numpy.ndarray`` of dtype object

        Dynamics likelihood mapping or 'transition model', mapping from hidden states at ``t`` to hidden states at ``t+1``, given some control state ``u``.

        Each element ``B[f]`` of this object array stores a 3-D tensor for hidden state factor ``f``, whose entries ``B[f][s, v, u]`` store the probability

        of hidden state level ``s`` at the current time, given hidden state level ``v`` and action ``u`` at the previous time.

    policy: 2D ``numpy.ndarray``

        Array that stores actions entailed by a policy over time. Shape is ``(num_timesteps, num_factors)`` where ``num_timesteps`` is the temporal

        depth of the policy and ``num_factors`` is the number of control factors.

    Returns

    -------

    qs_pi: ``list`` of ``numpy.ndarray`` of dtype object

        Predictive posterior beliefs over hidden states expected under the policy, where ``qs_pi[t]`` stores the beliefs about

        hidden states expected under the policy at time ``t``

    """

    n_steps = policy.shape[0]

    n_factors = policy.shape[1]

    # initialise posterior predictive density as a list of beliefs over time, including current posterior beliefs about hidden states as the first element

    qs_pi = [qs] + [utils.obj_array(n_factors) for t in range(n_steps)]

    

    # get expected states over time

    for t in range(n_steps):

        for control_factor, action in enumerate(policy[t,:]):

            qs_pi[t+1][control_factor] = B[control_factor][:,:,int(action)].dot(qs_pi[t][control_factor])

            qs_pi[t+1][1] = vision_matrix[qs_pi[t+1][0].argmax()]

            # # Debug visualization:
            # print(f"\n--- Debugging at timestep {t+1} and policy {policy[t,:]}")
            # print("Location beliefs:")
            # plot_beliefs(qs_pi[t+1][0], title_str=f"Location Beliefs - Step {t+1}")
            # print("Color beliefs:")
            # plot_beliefs(qs_pi[t+1][1], title_str=f"Color Beliefs - Step {t+1}")
            

    return qs_pi[1:]

In [47]:
from pymdp import control

# Inject our custom function
control.get_expected_states = custom_get_expected_states

# Active Inference

In [48]:
from pymdp.agent import Agent

In [49]:
agent_pos, redspots, goal_location

In [50]:
# Create base agent
my_agent = Agent(A=A, B=B, C=C, D=D, policy_len=3)



## Create Observation

In [51]:
grid_locations

In [52]:
def create_color_observation(position, red_obs, green_obs, white_obs):

    if red_obs != ['Null']:
        if position in red_obs: return 1  # RED
    if green_obs == position: return 2 # GREEN
    elif white_obs != ['Null']:
        if position in white_obs: return 0 # WHITE
    return 3

In [53]:
def create_observation(position, red_obs, green_obs, white_obs):
    return [grid_locations.index(position), create_color_observation(position, red_obs, green_obs, white_obs)]

## Coppelia Environment + Gridworld

In [54]:
from coppeliasim_zmqremoteapi_client import RemoteAPIClient
import numpy as np
import time
from math import comb

client = RemoteAPIClient()
sim = client.getObject('sim')

def move_to_grid(x, y):
    '''Moves coppelia coordinates (x,y) to a 40x40 grid, z coordinate remains constant, outputs coordinate in terms of grid'''
    
    # Translate x,y coordinate 2.5 up and 2.5 right
    x = x + 2.5
    y = y + 2.5
    
    # Ensure coordinates (x,y) are within (0,0) and (5,5)
    if x > 5 or x < 0:
        return "Invalid x coordinate!"
    elif y > 5 or y < 0:
        return "Invalid y coordinate!"
    
    # Convert x, y to grid indices by dividing by 0.05 (since each grid cell is 0.05 wide)
    x_grid = round(x / 0.125)
    y_grid = round(y / 0.125)
    
    # Ensure that the coordinates are within valid grid range (0 to 200)
    if x_grid > 40 or x_grid < 0:
        return "Invalid x grid point!"
    if y_grid > 40 or y_grid < 0:
        return "Invalid y grid point!"
    
    # Return the grid indices
    return (x_grid, y_grid)

    
def grid_to_coordinates(x_grid, y_grid):
    '''Converts a valid 200x200 grid point back into coppelia (x,y,z) coordinates in the range (x,y) = (0,0)-(5,5), z remains constant'''
    
    # Ensure the grid points are within valid range (0 to 200)
    if x_grid > 40 or x_grid < 0:
        return "Invalid x grid point!"
    if y_grid > 40 or y_grid < 0:
        return "Invalid y grid point!"
    
    # Reverse the grid index conversion by multiplying by 0.05
    x = x_grid * 0.125
    y = y_grid * 0.125
    
    # Return the original (x, y, z) coordinates
    return (x, y, 0.05)   

def get_object_position(object_name):
    # Step 2: Get the object handle by name
    objectHandle = sim.getObject(f'/{object_name}')
    
    if objectHandle == -1:
        raise Exception(f"Object '{object_name}' not found.")
    
    # Step 3: Get the position of the obstacle relative to the world (-1 means world reference)
    objectPosition = sim.getObjectPosition(objectHandle, -1)
    
    # Round each element in the position to the nearest thousandth
    roundedPosition = [round(coord, 3) for coord in objectPosition]
    
    print(f"Position of {object_name}: {roundedPosition}")
    return roundedPosition

def create_bounding_locations(position, dimensions):
    (x, y) = position
    (a, b, c) = dimensions

    # Bounding locations
    top_right = (x + a/2, y + b/2)
    bottom_left = (x - a/2, y - b/2)
    top_left = (x - a/2, y + b/2)
    bottom_right = (x + a/2, y - b/2)

    # Midpoints
    mid_top = ((top_right[0] + top_left[0]) / 2, (top_right[1] + top_left[1]) / 2)
    mid_bottom = ((bottom_right[0] + bottom_left[0]) / 2, (bottom_right[1] + bottom_left[1]) / 2)
    mid_left = ((top_left[0] + bottom_left[0]) / 2, (top_left[1] + bottom_left[1]) / 2)
    mid_right = ((top_right[0] + bottom_right[0]) / 2, (top_right[1] + bottom_right[1]) / 2)
    

    return top_right, bottom_left, top_left, bottom_right, mid_top, mid_bottom, mid_left, mid_right


def get_obstacle_locations():
    sceneObjects = [sim.getObjectAlias(obj) for obj in sim.getObjectsInTree(sim.handle_scene)]
    for i in sceneObjects:
        if "Obstacle" in i:
            obstacleHandle = sim.getObject(sim.getObjectAlias(i))
            obstaclePosition = get_object_position(sim.getObjectAlias(i))
            obstacleDimensions = sim.getObjectSize(obstacleHandle)
            obstacleBoundingLocations = create_bounding_locations(obstaclePosition, obstacleDimensions)
            return obstacleBoundingLocations
    
def check_bounds(loc):
    '''Checks if a location is within the bounds of the grid'''
    
    x, y, z = loc
    if x > 5 or x < 0:
        return None
    elif y > 5 or y < 0:
        return None
    else:
        return loc



bubbleRobHandle = sim.getObject('/bubbleRob')
ctrlPts = [[0.0,0.0,0.05,0.0,0.0,0.0,1.0]]
ctrlPts_flattened = [coord for point in ctrlPts for coord in point]



def monitor_position():
    pos1 = sim.getObjectPosition(bubbleRobHandle, -1)
    time.sleep(0.1)
    pos2 = sim.getObjectPosition(bubbleRobHandle, -1)
    print(f"Position change: {np.array(pos2) - np.array(pos1)}")
    


In [55]:

class CoppeliaEnv():
    def get_initial_object_properties(self):
        initial_pos = sim.getObjectPosition(self.bubbleRobHandle, -1)
        initial_orient = sim.getObjectQuaternion(self.bubbleRobHandle, -1)
        return initial_pos[2], initial_orient

    def __init__(self, redspots, starting_loc=move_to_grid(get_object_position('bubbleRob')[0], get_object_position('bubbleRob')[1]), goal=move_to_grid(get_object_position('Goal_Loc')[0],get_object_position('Goal_Loc')[1])):
        # Get robot handle first and ensure it's valid
        try:
            self.bubbleRobHandle = sim.getObject('/bubbleRob')  # Get handle directly here
            if self.bubbleRobHandle == -1:
                raise Exception("Could not get handle to bubbleRob")
        except Exception as e:
            print(f"Error getting robot handle: {e}")
            raise

        # Initialize coordinates 
        self.x, self.y = starting_loc
        self.init_loc = starting_loc
        self.current_location = (self.x, self.y)
        self.goal = goal
        self.redspots = redspots

        # Initialize observations
        self.red_obs = ['Null']
        self.green_obs = 'Null'
        self.white_obs = ['Null']
        
        # Initialize properties
        self.initial_z, self.initial_orientation = self.get_initial_object_properties()

        # Initialize path-related variables
        self.ctrlPts = ctrlPts
        self.ctrlPts_flattened = ctrlPts_flattened
        self.t = 0
        self.posAlongPath = 0
        self.velocity = 0.08
        self.totalLength = self.calculate_total_length()
        
        self.previousSimulationTime = 0
        self.agent_reward = 0


    def bezier_recursive(self, t):
        n = (len(self.ctrlPts) // 7) - 1
        point = np.zeros(3)
        total_weight = 0
        
        for i in range(n + 1):
            binomial_coeff = comb(n, i)
            weight = binomial_coeff * ((1 - t) ** (n - i)) * (t ** i)
            point_coords = np.array(self.ctrlPts[i * 7:i * 7 + 3])
            point += weight * point_coords
            total_weight += weight
        
        if total_weight > 0:
            point = point / total_weight
        
        point[2] = self.initial_z  # Now this will work as initial_z is set before this is called
        return point

    def calculate_total_length(self, subdivisions=1000):
        total_length = 0.0
        prev_point = self.bezier_recursive(0)
        
        for i in range(1, subdivisions + 1):
            t = i / subdivisions
            curr_point = self.bezier_recursive(t)
            total_length += np.linalg.norm(curr_point - prev_point)
            prev_point = curr_point
        
        # return total_length
        return 0.125

    def get_point_and_tangent(self, t):
        point = self.bezier_recursive(t)
        
        delta = 0.001
        t_next = min(1.0, t + delta)
        next_point = self.bezier_recursive(t_next)
        
        tangent = next_point - point
        if np.linalg.norm(tangent) > 0:
            tangent = tangent / np.linalg.norm(tangent)
        
        return point, tangent

    def update_orientation(self, current_pos, tangent):
        forward = np.array([1, 0, 0])  # Assuming robot's forward direction is along x-axis
        rotation_axis = np.cross(forward, tangent)
        
        if np.linalg.norm(rotation_axis) > 0:
            rotation_axis = rotation_axis / np.linalg.norm(rotation_axis)
            angle = np.arccos(np.dot(forward, tangent))
            
            s = np.sin(angle/2)
            quat = [np.cos(angle/2), rotation_axis[0]*s, rotation_axis[1]*s, rotation_axis[2]*s]
            
            ssim.setObjectQuaternion(self.bubbleRobHandle, -1, quat)

    def follow_path(self):
        # Verify handle is valid before starting
        if self.bubbleRobHandle == -1:
            print("Invalid robot handle")
            return
            
        self.previousSimulationTime = sim.getSimulationTime()
        start_pos = sim.getObjectPosition(self.bubbleRobHandle, -1)
        print(f"Starting position: {start_pos}")  # Debug print
        
        while self.posAlongPath < self.totalLength:
            t = sim.getSimulationTime()
            deltaT = t - self.previousSimulationTime
            
            if deltaT <= 0.0:
                self.previousSimulationTime = t
                continue
            
            self.posAlongPath += self.velocity * deltaT
            
            if self.posAlongPath >= self.totalLength - 0.001:
                self.posAlongPath = self.totalLength
                print("Reached the end of the path!")
                break
            
            t_norm = np.clip(self.posAlongPath / self.totalLength, 0, 1)
            current_pos, tangent = self.get_point_and_tangent(t_norm)
            
            # Ensure Z coordinate and convert to list
            current_pos[2] = self.initial_z
            pos_list = current_pos.tolist()
            
            # Set position with error checking
            try:
                ret = sim.setObjectPosition(self.bubbleRobHandle, -1, pos_list)
                if ret == -1:
                    print(f"Failed to set position: {pos_list}")
                current_actual_pos = sim.getObjectPosition(self.bubbleRobHandle, -1)
                if current_actual_pos != pos_list:
                    print(f"Position mismatch - Requested: {pos_list}, Actual: {current_actual_pos}")
            except Exception as e:
                print(f"Error setting position: {e}")
                break
                
            # Update orientation
            self.update_orientation(current_pos, tangent)
            
            self.previousSimulationTime = t
            sim.step()
            time.sleep(0.05)
            
        # Print final position for debugging
        end_pos = sim.getObjectPosition(self.bubbleRobHandle, -1)
        print(f"Ending position: {end_pos}")

    
    def step(self, action_label):
        sim.startSimulation()
        if action_label == "UP": 
            self.y = max(0, self.y - 1)  # Move up (decrease y)
            self.ctrlPts.append([grid_to_coordinates(self.x,self.y)[0], grid_to_coordinates(self.x,self.y)[1], 0.05,0.0,0.0,0.0,1.0])
            self.ctrlPts_flattened = [coord for point in ctrlPts for coord in point]
            self.totalLength = self.calculate_total_length()
            print(self.totalLength)
            print(self.ctrlPts)
            print(self.ctrlPts_flattened)
            pathHandle = sim.createPath(
                self.ctrlPts_flattened,
                0,  # Options: open path
                100,  # Subdivision for smoothness
                1,  # No smoothness
                0,  # Orientation mode
                [0.0, 0.0, 1.0]
            )
            self.follow_path()
            monitor_position()
            time.sleep(0.05)
            self.ctrlPts.clear()
            
        elif action_label == "DOWN": 
            self.y = min(grid_dims[1] - 1, self.y + 1)  # Move down (increase y)
            self.ctrlPts.append([grid_to_coordinates(self.x,self.y)[0], grid_to_coordinates(self.x,self.y)[1], 0.05,0.0,0.0,0.0,1.0])
            self.ctrlPts_flattened = [coord for point in ctrlPts for coord in point]
            self.totalLength = self.calculate_total_length()
            print(self.totalLength)
            print(self.ctrlPts)
            print(self.ctrlPts_flattened)
            pathHandle = sim.createPath(
                self.ctrlPts_flattened,
                0,  # Options: open path
                100,  # Subdivision for smoothness
                1,  # No smoothness
                0,  # Orientation mode
                [0.0, 0.0, 1.0]
            )
            self.follow_path()
            monitor_position()
            time.sleep(0.05)
            self.ctrlPts.clear()
            
        elif action_label == "LEFT": 
            self.x = max(0, self.x - 1)  # Move left (decrease x)
            self.ctrlPts.append([grid_to_coordinates(self.x,self.y)[0], grid_to_coordinates(self.x,self.y)[1], 0.05,0.0,0.0,0.0,1.0])
            self.ctrlPts_flattened = [coord for point in ctrlPts for coord in point]
            self.totalLength = self.calculate_total_length()
            print(self.totalLength)
            print(self.ctrlPts)
            print(self.ctrlPts_flattened)
            pathHandle = sim.createPath(
                self.ctrlPts_flattened,
                0,  # Options: open path
                100,  # Subdivision for smoothness
                1,  # No smoothness
                0,  # Orientation mode
                [0.0, 0.0, 1.0]
            )
            self.follow_path()
            monitor_position()
            time.sleep(0.05)
            self.ctrlPts.clear()
            self.ctrlPts.append([grid_to_coordinates(self.x,self.y)[0], grid_to_coordinates(self.x,self.y)[1], 0.05,0.0,0.0,0.0,1.0])
        elif action_label == "RIGHT": 
            self.x = min(grid_dims[0] - 1, self.x + 1)  # Move right (increase x)
            self.ctrlPts.append([grid_to_coordinates(self.x,self.y)[0], grid_to_coordinates(self.x,self.y)[1], 0.05,0.0,0.0,0.0,1.0])
            self.ctrlPts_flattened = [coord for point in ctrlPts for coord in point]
            self.totalLength = self.calculate_total_length()
            print(self.totalLength)
            print(self.ctrlPts)
            print(self.ctrlPts_flattened)
            pathHandle = sim.createPath(
                self.ctrlPts_flattened,
                0,  # Options: open path
                100,  # Subdivision for smoothness
                1,  # No smoothness
                0,  # Orientation mode
                [0.0, 0.0, 1.0]
            )
            self.follow_path()
            monitor_position()
            time.sleep(0.05)
            self.ctrlPts.clear()
            self.ctrlPts.append([grid_to_coordinates(self.x,self.y)[0], grid_to_coordinates(self.x,self.y)[1], 0.05,0.0,0.0,0.0,1.0])
        # Update current_location tuple after movement
        self.current_location = (self.x, self.y)
        print(f"self.current_location: {self.current_location}")
        
        # Update vision with current coordinates
        self.vision = update_vision(self.current_location, grid_dims, 6)

        self.loc_obs = self.current_location

        # Reset observations at each step
        self.red_obs = ['Null']
        self.white_obs = ['Null']
        self.green_obs = 'Null'

        # Update observations based on vision
        for spot in self.vision:
            if spot in self.redspots:
                if 'Null' in self.red_obs:
                    self.red_obs = [spot]
                else:
                    self.red_obs.append(spot)
            elif spot == self.goal:
                self.green_obs = spot
                
            else:
                if 'Null' in self.white_obs:
                    self.white_obs = [spot]
                else:
                    self.white_obs.append(spot)

        # Update rewards and observations based on current location
        if self.current_location in self.redspots:
            self.agent_reward -= 5
            if 'Null' in self.red_obs:
                self.red_obs = [self.current_location]
            else:
                self.red_obs.append(self.current_location)
        elif self.current_location == self.goal:
            self.agent_reward += 20
            self.green_obs = self.current_location
        else:
            if 'Null' in self.white_obs:
                self.white_obs = [self.current_location]
            else:
                self.white_obs.append(self.current_location)
        
        return self.loc_obs, self.green_obs, self.white_obs, self.red_obs, self.agent_reward
    
    def reset(self):
        self.x, self.y = self.init_loc
        self.current_location = (self.x, self.y)
        print(f'Re-initialized location to {self.current_location}')
        self.loc_obs = self.current_location
        self.green_obs, self.white_obs, self.red_obs, self.agent_reward = 'Null', ['Null'], ['Null'], 0

        return self.loc_obs, self.green_obs, self.white_obs, self.red_obs, self.agent_reward
    
my_env = CoppeliaEnv(starting_loc = agent_pos, redspots=redspots, goal = goal_location)
loc_obs, green_obs, white_obs, red_obs, agent_reward = my_env.reset()
loc_obs, green_obs, white_obs, red_obs, agent_reward
loc_obs, green_obs, white_obs, red_obs, agent_reward = my_env.step('STAY')

## Loop

In [56]:
obs = create_observation(loc_obs, red_obs, green_obs, white_obs)

In [ ]:
history_of_locs = [loc_obs]

T = 15


for t in range(T):
    obs = create_observation(loc_obs, red_obs, green_obs, white_obs)
    print(f"Observation: {obs}")
    vision_matrix = update_vision_matrix(vision_matrix, obs)

    # Update beliefs
    qs = my_agent.infer_states(obs)

    # plot_beliefs(qs[0])
    # plot_beliefs(qs[1])
    
    # Update obs matrix
    surrounding_locs = update_vision(loc_obs, grid_dims, 2)
    for loc in surrounding_locs:
        obs = create_observation(loc, red_obs, green_obs, white_obs)
        vision_matrix = update_vision_matrix(vision_matrix, obs)
        
    visualize_vision_matrix(vision_matrix, grid_dims)
    
    # Policy selection
    my_agent.infer_policies()
    chosen_action_id = my_agent.sample_action()
    
    movement_id = int(chosen_action_id[0])
    choice_action = actions[movement_id]
    print(f'Action at time {t}: {choice_action}')
    
    # Environment step
    loc_obs, green_obs, white_obs, red_obs, agent_reward = my_env.step(choice_action)
    
    print(agent_reward, loc_obs, green_obs, white_obs, red_obs)
    history_of_locs.append(loc_obs)
    print(f'Grid location at time {t}: {loc_obs}')
    print(f'Reward at time {t}: {agent_reward}')